In [19]:
LEDGER_RPC = "https://4c7f-209-129-88-187.ngrok-free.app"


In [18]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [2]:
# Chunk 1: Setup
!pip install -q pandas requests folium

import pandas as pd, requests, re, json, datetime as dt

# Set your ledger endpoint (change this to your public URL or use localhost for local testing)
LEDGER_RPC = "https://4c7f-209-129-88-187.ngrok-free.app"  # <-- Update this URL when available
SUBMIT_URL  = f"{LEDGER_RPC}/submit"
QUERY_URL   = f"{LEDGER_RPC}/query"

print("Ledger Endpoint:", LEDGER_RPC)


Ledger Endpoint: https://4c7f-209-129-88-187.ngrok-free.app


In [24]:
# Chunk 2: Load your CSV files (ensure they're uploaded to Colab)
tweets = pd.read_csv("final_social_zones.csv")
sensors = pd.read_csv("final_sensors_zones.csv")

# Convert the timestamp columns to datetime objects
tweets["timestamp"] = pd.to_datetime(tweets["timestamp"])
sensors["timestamp"] = pd.to_datetime(sensors["timestamp"])

print("Tweets shape:", tweets.shape)
print("Sensors shape:", sensors.shape)



Tweets shape: (50000, 7)
Sensors shape: (33409, 10)


In [25]:
tweets.head()

,user_id,text,timestamp,latitude,longitude,clean_text,zone_id
0,148149,fire in downtown,2023-01-01 00:00:00,37.891326,-122.119823,fire in downtown,18.0
1,133822,fire in downtown,2023-01-01 00:01:00,37.261545,-121.592279,fire in downtown,13.0
2,117131,emergency in Zone A,2023-01-01 00:02:00,37.976165,-122.425660,emergency in zone a,1.0
3,197271,flood near river,2023-01-01 00:03:00,37.217171,-121.747097,flood near river,4.0
4,105884,earthquake felt,2023-01-01 00:04:00,37.812096,-121.624895,earthquake felt,6.0


In [26]:
# Chunk 3: Extract hazard keywords from tweets
HAZARD_PATTERNS = {
    "fire":       r"\bfire\b",
    "earthquake": r"\bearthquake\b|\bquake\b",
    "flood":      r"\bflood\b",
    "evacuate":   r"\bevacuate\b|\bevacuation\b|\bemergency\b"
}

def extract_hazard(text):
    for hazard, pattern in HAZARD_PATTERNS.items():
        if re.search(pattern, str(text), flags=re.I):
            return hazard
    return "unknown"

# Assume your tweet text is in column 'clean_text'
tweets["tweet_hazard"] = tweets["clean_text"].apply(extract_hazard)
print("Hazard types found:", tweets["tweet_hazard"].unique())
tweets.head()


Hazard types found: ['fire' 'evacuate' 'flood' 'earthquake']


,user_id,text,timestamp,latitude,longitude,clean_text,zone_id,tweet_hazard
0,148149,fire in downtown,2023-01-01 00:00:00,37.891326,-122.119823,fire in downtown,18.0,fire
1,133822,fire in downtown,2023-01-01 00:01:00,37.261545,-121.592279,fire in downtown,13.0,fire
2,117131,emergency in Zone A,2023-01-01 00:02:00,37.976165,-122.425660,emergency in zone a,1.0,evacuate
3,197271,flood near river,2023-01-01 00:03:00,37.217171,-121.747097,flood near river,4.0,flood
4,105884,earthquake felt,2023-01-01 00:04:00,37.812096,-121.624895,earthquake felt,6.0,earthquake


In [33]:
# Chunk 4: Define decision logic for validating a tweet based on sensor readings

VALID_THRESHOLD = 60    # Sensor severity threshold on scale 1-100
TIME_WINDOW = "20min"   # ±10 minutes time window
MIN_ACTIVE = 1
HIGH_PCT_REQ = 20       # % of sensors high enough to consider the hazard real
MEAN_REQ = 20

def map_hazard_to_sensors(hazard):
    mapping = {
        "flood":      ["flood", "humidity"],
        "fire":       ["temp", "humidity"],
        "earthquake": ["seismic"],
        "evacuate":   ["flood", "temp", "seismic", "humidity"],
    }
    return mapping.get(hazard, [])

def validate_tweet(row):
    if row["tweet_hazard"] == "unknown":
        return "ignore", 0, 0, 0

    z, h, t0 = row["zone_id"], row["tweet_hazard"], row["timestamp"]
    # Select sensor readings from the same zone, sensor type relevant to the hazard,
    # and whose timestamp is within ± TIME_WINDOW.
    window = sensors[
        (sensors["zone_id"] == z) &
        (sensors["sensor_type"].isin(map_hazard_to_sensors(h))) &
        (abs(sensors["timestamp"] - t0) <= pd.Timedelta(TIME_WINDOW))
    ]

    if window.empty:
        return "fake", 0, 0, 0

    mean_read = window["reading_value"].mean()
    pct_high = (window["reading_value"] >= VALID_THRESHOLD).mean() * 100
    n_active = (window["status"].str.lower() == "active").sum()  # or window.status if only active in window

    if pct_high >= HIGH_PCT_REQ and mean_read >= MEAN_REQ and n_active >= MIN_ACTIVE:
        verdict = "valid"
    elif pct_high < 5:
        verdict = "fake"
    else:
        verdict = "uncertain"

    return verdict, pct_high, mean_read, n_active

# Apply validation to each tweet and save the output in new columns
results = tweets.apply(lambda r: pd.Series(validate_tweet(r),
                                             index=["validation_status", "pct_high", "mean_read", "n_active"]), axis=1)
tweets = pd.concat([tweets, results], axis=1)

print(tweets["validation_status"].value_counts())
tweets.head()

validation_status
fake         43493
valid         3808
uncertain     2699
Name: count, dtype: int64


,user_id,text,timestamp,latitude,longitude,clean_text,zone_id,tweet_hazard,status,pct_high,...,mean_read,n_active,status,pct_high,mean_read,n_active,validation_status,pct_high,mean_read,n_active
0,148149,fire in downtown,2023-01-01 00:00:00,37.891326,-122.119823,fire in downtown,18.0,fire,fake,0.0,...,0.000000,0,fake,0.0,7.713270,1,fake,0.0,7.713270,1
1,133822,fire in downtown,2023-01-01 00:01:00,37.261545,-121.592279,fire in downtown,13.0,fire,fake,0.0,...,0.000000,0,fake,0.0,0.000000,0,fake,0.0,0.000000,0
2,117131,emergency in Zone A,2023-01-01 00:02:00,37.976165,-122.425660,emergency in zone a,1.0,evacuate,fake,0.0,...,0.000000,0,fake,0.0,0.000000,0,fake,0.0,0.000000,0
3,197271,flood near river,2023-01-01 00:03:00,37.217171,-121.747097,flood near river,4.0,flood,fake,0.0,...,20.295672,1,fake,0.0,20.295672,1,fake,0.0,20.295672,1
4,105884,earthquake felt,2023-01-01 00:04:00,37.812096,-121.624895,earthquake felt,6.0,earthquake,fake,0.0,...,0.000000,0,fake,0.0,0.000000,0,fake,0.0,0.000000,0


In [30]:
# Chunk 5: Push each record (e.g. with status 'valid', 'uncertain', or 'fake')
def push_tx(row):
    tx = {
        "tweet_id":  str(row["user_id"]),
        "zone_id":   row["zone_id"],
        "hazard":    row["tweet_hazard"],
        "status":    row["status"],
        "timestamp": row["timestamp"].isoformat(),
        "pct_high":  row["pct_high"],
        "mean_read": row["mean_read"],
        "n_active":  int(row["n_active"]),
    }
    # Disable SSL verification for testing
    response = requests.post(SUBMIT_URL, json=tx, timeout=10, verify=False)
    return response.status_code

# Choose tweets with a definitive status (exclude "ignore")
mask = tweets["status"].isin(["valid", "uncertain", "fake"])
push_results = tweets[mask].apply(push_tx, axis=1)
print("Pushed", (push_results == 200).sum(), "records to the ledger")


ValueError: cannot reindex on an axis with duplicate labels

In [17]:
# Chunk 6: Query a sample record to verify it was stored
sample_id = str(tweets[mask].iloc[0]["user_id"])
print("Querying for tweet_id:", sample_id)
query_resp = requests.get(f"{QUERY_URL}/{sample_id}", timeout=10)
print("Response:", query_resp.json())


Querying for tweet_id: 148149
Response: {'hazard': 'fire', 'mean_read': 0.0, 'n_active': 0, 'pct_high': 0.0, 'status': 'fake', 'timestamp': '2023-01-01T00:00:00', 'tweet_id': '148149', 'zone_id': 18.0}


In [23]:
import datetime as dt
import pandas as pd   # ensure pandas is imported
import requests     # ensure requests is imported

def process_input_tweet():
    # Prompt user for tweet details
    text = input("Enter tweet text: ")
    zone_id = input("Enter zone (e.g., ZoneA): ")
    ts_input = input("Enter tweet timestamp (YYYY-MM-DD HH:MM:SS): ")

    # Convert the timestamp string to a datetime object using pandas
    try:
        tweet_timestamp = pd.to_datetime(ts_input)
    except Exception as e:
        print("Invalid timestamp format:", e)
        return

    # Explicitly extract the hazard from the tweet text
    tweet_hazard = extract_hazard(text)

    # Build the tweet dictionary from input, including the extracted hazard
    tweet = {
        "text": text,
        "zone_id": zone_id,
        "timestamp": tweet_timestamp,
        "tweet_hazard": tweet_hazard
    }

    # Validate the tweet using your custom logic.
    # validate_tweet returns a tuple: (verdict, pct_high, mean_read, n_active)
    verdict, pct_high, mean_read, n_active = validate_tweet(tweet)

    # Calculate the number of sensors in the matching time window (if not provided by validate_tweet)
    TIME_WINDOW = "10min"  # Same time window used in validation logic
    window = sensors[
        (sensors["zone_id"] == zone_id) &
        (sensors["sensor_type"].isin(map_hazard_to_sensors(tweet_hazard))) &
        (abs(sensors["timestamp"] - tweet_timestamp) <= pd.Timedelta(TIME_WINDOW))
    ]
    num_sensors = len(window)

    # Display the validation results
    print("\nValidation Result:")
    print("Tweet Hazard         :", tweet_hazard)
    print("Verdict              :", verdict)
    print("Mean Sensor Reading  :", mean_read)
    print("Percent High Readings:", pct_high)
    print("Active Sensors       :", n_active)
    print("Sensors in Window    :", num_sensors)

    # Build payload (generate a unique tweet_id using current datetime)
    payload = {
        "tweet_id": "manual-" + dt.datetime.now().strftime("%Y%m%d%H%M%S"),
        "zone_id": zone_id,
        "hazard": tweet_hazard,
        "status": verdict,
        "timestamp": tweet_timestamp.isoformat(),
        "pct_high": pct_high,
        "mean_read": mean_read,
        "n_active": n_active
    }

    # Send the payload to the blockchain ledger (via your Flask API)
    # (For testing, you may use verify=False to bypass SSL verification issues)
    response = requests.post(SUBMIT_URL, json=payload, timeout=10, verify=False)
    if response.status_code == 200:
        print("\nTweet successfully sent to the blockchain ledger.")
    else:
        print("\nError sending tweet to the ledger. Status:", response.status_code)
        print(response.text)

# Run the function to process a tweet input in real time.
process_input_tweet()


Enter tweet text: fire in la
Enter zone (e.g., ZoneA): 8
Enter tweet timestamp (YYYY-MM-DD HH:MM:SS): 2023-01-01 00:30:00

Validation Result:
Tweet Hazard         : fire
Verdict              : fake
Mean Sensor Reading  : 0
Percent High Readings: 0
Active Sensors       : 0
Sensors in Window    : 0

Tweet successfully sent to the blockchain ledger.
